# ChatBot RAG com documentos PDF
---
Este notebook cria o RAG para o contexto do chatbot.

### IMPORTS

In [33]:
import os
import time
from dotenv import load_dotenv
from typing import Dict, Any, List, Optional

# Groq
from langchain_groq import ChatGroq

## core
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.documents import Document

# pypdf e vectordb
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS

# embeddings e textsplitters
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

import warnings
warnings.filterwarnings("ignore")

### CONFIGURAÇÃO DA API

In [2]:
load_dotenv()

GROQ_API_KEY = os.environ["GROQ_API_KEY"]

if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY não foi configurada!")

print("API Key configurada!")

API Key configurada!


### CONFIGURAÇÕES GLOBAIS

In [81]:
GROQ_MODELS = {
    "llama-3.3-70b": "llama-3.3-70b-versatile",
    "llama-3.1-8b": "llama-3.1-8b-instant",
    "kimi-k2-instruct": "moonshotai/kimi-k2-instruct",
    "gpt-oss-20b": "openai/gpt-oss-20b",
    "qwen3-32b": "qwen/qwen3-32b"
}
CONFIGS = {
    "model":GROQ_MODELS['gpt-oss-20b'],
    "temperature": 0.1,
    "max_tokens": 2048,
    "max_messages":20,
    "chunk_size":1000,
    "chunk_overlap":200,
    "embeddings_model":["sentence_transformers/all-MiniLM-L6-v2", "sentence-transformers/paraphrase-MiniLM-L6-v2"],
    "retriever_k":3,
    "retriever_fetch_k":6
}

print(f"""
CONFIGURAÇÕES
    - Modelo LLM              - {CONFIGS['model']}
    - Temperatura             - {CONFIGS['temperature']}
    - Max tokens              - {CONFIGS['max_tokens']}
    - Max mensagens histórico - {CONFIGS['max_messages']}
    - Modelo Embeedings:      - {CONFIGS['embeddings_model'][1]}
""")


CONFIGURAÇÕES
    - Modelo LLM              - openai/gpt-oss-20b
    - Temperatura             - 0.1
    - Max tokens              - 2048
    - Max mensagens histórico - 20
    - Modelo Embeedings:      - sentence-transformers/paraphrase-MiniLM-L6-v2



### FUNÇÕES AUXILIARES

In [23]:
# check conexão
def test_groq_connection() -> bool:
    """Testa a conexão com a API Groq."""
    try:
        llm = ChatGroq(
            model=CONFIGS['model'],
            temperature=0,
            api_key=GROQ_API_KEY
        )
        res = llm.invoke('Responda apenas: OK')
        if res.content.strip().upper() == 'OK':
            return True
        else:
            return False
    except Exception as e:
        print(f'Erro: {str(e)}')
        return False

### CLASSE PRINCIPAL

In [90]:
class RAGChatBot:
    """
    ChatBot para conversar através da API Groq
    """

    def __init__(self, configs:Dict[str,Any]=CONFIGS):
        """Inicializa o chatbot"""
        self.configs = configs
        self.llm = None
        self.history:List[Any] = []

        # RAG
        self.embeddings = None
        self.vector_store = None
        self.retriever = None
        self.documents_loaded = False

        print('=== INICIALIZANDO CHATBOT ===')
        self._initialize_llm()
        self._initialize_system_prompt()
        self._initialize_embeddings()
        print('\nChatBot pronto!')

    def _initialize_llm(self):
        """Inicializa o modelo de linguagem Groq"""
        self.llm = ChatGroq(
            model=self.configs['model'],
            temperature=self.configs['temperature'],
            max_tokens=self.configs['max_tokens'],
            api_key=GROQ_API_KEY
        )
        print(f'Modelo {self.configs["model"]} carregado!')

    def _initialize_system_prompt(self):
        """Mensagem fixa do sistema"""
        system_message = SystemMessage(
            content="""
            Você é um assistente prestativo especializado em responder perguntas com base no contexto fornecido.
            Use as informações dos documentos para dar respostas precisas e objetivas. 
            Se não souber a resposta com base no contexto, diga isso honestamente.
            """
        )
        self.history.append(system_message)

    def _initialize_embeddings(self):
        """Inicializa o modelo de embeedings"""
        print(f"Carregando embeddings: {self.configs['embeddings_model'][1]}...")
        self.embeddings = HuggingFaceEmbeddings(
            model_name=self.configs['embeddings_model'][1],
            model_kwargs={'device':'cpu'},
            encode_kwargs={'normalize_embeddings':True},
        )
        print('Embeddings carregados!')
    
    def load_pdf(self, pdf_path: str, save_index: bool = True, index_path: str = 'faiss_index') -> bool:
        """
        Carrega um PDF e cria o índice vetorial
        """
        try:
            print(f'Carregando PDF: {pdf_path}')
            
            if os.path.exists(index_path) and os.path.isdir(index_path): # persistido?
                print(f'Índice encontrado em {index_path}')
                self.vector_store = FAISS.load_local(
                    index_path,
                    self.embeddings,  
                    allow_dangerous_deserialization=True                    
                )
                print('Índice carregado do disco')
            else:
                loader = PyMuPDFLoader(pdf_path)
                documents = loader.load()
                print(f'PDF carregado com {len(documents)} páginas')
    
                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=self.configs['chunk_size'],
                    chunk_overlap=self.configs['chunk_overlap'],
                    length_function=len,
                    separators=["\n\n", "\n", ".", "!", "?", ";", " ", ""]
                )
                chunks = text_splitter.split_documents(documents)
                print(f'Texto dividido em {len(chunks)} chunks')
    
                print('Criando índice vetorial')
                self.vector_store = FAISS.from_documents(chunks, self.embeddings)
                print('Índice vetorial criado!')
    
                if save_index:
                    self.vector_store.save_local(index_path)
                    print(f'Índice salvo em {index_path}')
    
            # retriever
            self.retriever = self.vector_store.as_retriever(
                search_type='mmr',
                search_kwargs={
                    'k': self.configs['retriever_k'],
                    'fetch_k': self.configs['retriever_fetch_k']
                }
            )
            self.documents_loaded = True  
            print('Sistema RAG pronto para uso!')
            return True
                    
        except Exception as e:
            print(f'Erro ao carregar PDF: {str(e)}')
            return False

    def retrieve_context(self, query:str) -> str:
        """
        Recupera o contexto relevante dos documentos.

        Args:
            query: pergunta do usuario
        Returns:
            string com o contexto concatenado
        """
        if not self.documents_loaded or not self.retriever:
            return ''
        
        docs = self.retriever.invoke(query)
        if not docs:
            return ''

        context_parts=[]
        for i, doc in enumerate(docs,1):
            source = doc.metadata.get('source', 'Documento')
            page = doc.metadata.get('page', 'N/A')
            context_parts.append(f'[Documento {i} - Página {page}]:\n{doc.page_content}')
        
        return '\n\n'.join(context_parts)

    def chat(self, user_input: str, use_rag: bool = True, verbose=True):
        """
        Processa um pergunta do usuário e retorna a resposta
        """
        start_time = time.time()
    
        # contexto
        if use_rag and self.documents_loaded:
            context = self.retrieve_context(user_input)
            if context:
                enhanced_prompt = f"""Com base no seguinte contexto, responda à pergunta do usuário.
    Se a resposta não estiver no contexto, diga que não possui essa informação.
    
    === CONTEXTO ===
    {context}
    
    === PERGUNTA DO USUÁRIO ===
    {user_input}
    
    === INSTRUÇÃO ===
    Responda de forma objetiva usando apenas as informações do contexto acima."""
                
                message = HumanMessage(content=enhanced_prompt)
                if verbose:
                    print(f'Contexto recuperado: {len(context)} caracteres')
            else:
                message = HumanMessage(content=user_input)
                if verbose:
                    print('Sem contexto relevante.')
        else:
            # sem RAG pergunta direta
            message = HumanMessage(content=user_input)
    
        # apendar histórico
        self.history.append(message)
        
        # gerar e apendar resposta
        res = self.llm.invoke(self.history)
        self.history.append(AIMessage(content=res.content))
    
        # gerenciar histórico
        if len(self.history) > self.configs['max_messages']:
            self.history = [self.history[0]] + self.history[-(self.configs['max_messages']-1):]
    
        elapsed_time = time.time() - start_time
    
        # output
        if verbose:
            print(f'\nUSER: {user_input}')
            print(f'\nBOT: {res.content}')
            print(f'\nTempo: {elapsed_time:.2f}s | Histórico: {len(self.history)} mensagens')
            return None
        else: return res.content
        #     print(res.content)
        
        # return res.content
    
    
    def clear_history(self):
        """Limpa o histórico de conversação mantendo o systemp prompt"""
        system_msg = self.history[0] if self.history else None
        self.history = [system_msg] if system_msg else []
        print('Histórico limpo')

    def clear_documents(self):
        """Remove todos os documentos carregados"""
        self.vector_store=None
        self.retriever=None
        self.documents_loaded=False
        print('Documentos resolvidos')

    def get_stats(self):
        """Retorna estatísticas do sistema RAG"""
        stats={
            'documentos_carregados':self.documents_loaded,
            'mensagens_no_historico':len(self.history),
            'modelo_llm': self.configs['model'],
            'modelo_embeddings': self.configs['embeddings_model'][1]
        }
        if self.vector_store:
            stats['documentos_no_indice']= self.vector_store.index.ntotal
        return stats

### INICIALIZAR CHATBOT

In [62]:
if test_groq_connection():
    bot = RAGChatBot()

=== INICIALIZANDO CHATBOT ===
Modelo llama-3.1-8b-instant carregado!
Carregando embeddings: sentence-transformers/paraphrase-MiniLM-L6-v2...
Embeddings carregados!

ChatBot pronto!


### FAZER PERGUNTAS

#### CHAT

In [63]:
bot.chat('Qual a capital da França')


USER: Qual a capital da França

BOT: A capital da França é Paris.

Tempo: 0.23s | Histórico: 3 mensagens


In [64]:
bot.chat('Resuma econometria em uma frase.')


USER: Resuma econometria em uma frase.

BOT: A econometria é a aplicação de métodos estatísticos e matemáticos para analisar e prever fenômenos econômicos, permitindo a tomada de decisões informadas em áreas como política econômica, gestão de empresas e planejamento.

Tempo: 1.57s | Histórico: 5 mensagens


In [65]:
bot.chat('Explique machine learning em uma linha e somente com emojis',verbose=False)

'🤖💻📊📈💡🔍'

In [66]:
bot.chat('qual foi minha primeira pergunta?')


USER: qual foi minha primeira pergunta?

BOT: Sua primeira pergunta foi sobre a capital da França.

Tempo: 0.41s | Histórico: 9 mensagens


In [67]:
bot.clear_history()

Histórico limpo


In [68]:
bot.chat('qual foi minha primeira pergunta?', verbose=False)

'Essa é a sua primeira pergunta. Você perguntou: "qual foi minha primeira pergunta?"'

In [74]:
bot.get_stats()

{'documentos_carregados': False,
 'mensagens_no_historico': 3,
 'modelo_llm': 'llama-3.1-8b-instant',
 'modelo_embeddings': 'sentence-transformers/paraphrase-MiniLM-L6-v2'}

#### CHAT COM RAG

In [91]:
bot_rag = RAGChatBot()

=== INICIALIZANDO CHATBOT ===
Modelo openai/gpt-oss-20b carregado!
Carregando embeddings: sentence-transformers/paraphrase-MiniLM-L6-v2...
Embeddings carregados!

ChatBot pronto!


In [92]:
# carregar pdf
bot_rag.load_pdf("C:\\Users\\wsant\\Downloads\\O'Reilly - PT - SQL Guia Prático - Alice Zhao.pdf")

Carregando PDF: C:\Users\wsant\Downloads\O'Reilly - PT - SQL Guia Prático - Alice Zhao.pdf
Índice encontrado em faiss_index
Índice carregado do disco
Sistema RAG pronto para uso!


True

In [94]:
bot_rag.chat('Qual o principal tema deste documento?')

Contexto recuperado: 2241 caracteres

USER: Qual o principal tema deste documento?

BOT: O principal tema do material apresentado é **SQL e operações em banco de dados**, abordando consultas, funções de data/hora, manipulação de tabelas e conceitos avançados de execução de consultas.

Tempo: 0.64s | Histórico: 3 mensagens


In [95]:
bot_rag.chat("Quais são as principais conclusões ou recomendações?")

Contexto recuperado: 1821 caracteres

USER: Quais são as principais conclusões ou recomendações?

BOT: **Principais conclusões / recomendações extraídas do contexto**

1. **Estrutura e legibilidade do SQL**  
   - Use as cláusulas padrão (SELECT, FROM, WHERE, GROUP BY, HAVING, ORDER BY).  
   - A capitalização e espaços em branco não afetam a execução, mas boas práticas de formatação melhoram a legibilidade.

2. **Manuseio de colunas**  
   - Evite depender de nomes de colunas fixos em código de produção, pois alterações na estrutura da tabela podem quebrar o código.  
   - Prefira selecionar expressões (ou aliases) que sejam robustas a mudanças de esquema.

3. **Agrupamento e agregação**  
   - Quando precisar de resultados semelhantes a `GROUP BY`, considere usar funções de janela (`OVER … PARTITION BY …`) com `FIRST_VALUE` como alternativa.

4. **Concatenação de texto**  
   - Para combinar valores de múltiplas colunas em uma única coluna, use a função `CONCAT` ou o operador de conc

In [96]:
bot_rag.chat("Fale sobre o comando SELECT")

Contexto recuperado: 2175 caracteres

USER: Fale sobre o comando SELECT

BOT: **Comando SELECT – pontos principais extraídos do contexto**

| Tema | O que o contexto diz |
|------|----------------------|
| **Função básica** | `SELECT` é usado para listar colunas ou expressões de uma tabela ou de uma sub‑consulta. Ex.: `SELECT name, MAX(stop) as num_stops FROM tour GROUP BY name`. |
| **Agrupamento** | Pode ser combinado com `GROUP BY` e funções agregadas (`MAX`, `AVG`, etc.). Ex.: `SELECT MAX(num_orders) FROM my_cte`. |
| **Sub‑consultas** | `SELECT` pode ser usado dentro de outra consulta, como em `SELECT AVG(num_stops) FROM (SELECT name, MAX(stop) …) tour_stops`. |
| **Junções** | `SELECT` pode combinar dados de duas ou mais tabelas usando `JOIN`. Ex.: `SELECT * FROM states s INNER JOIN pets p ON s.name = p.name`. |
| **Filtros e ordenação** | Embora não detalhados explicitamente, o contexto indica que `SELECT` pode ser usado com cláusulas `WHERE`, `ORDER BY`, etc., para refinar resu

In [97]:
bot_rag.get_stats()

{'documentos_carregados': True,
 'mensagens_no_historico': 7,
 'modelo_llm': 'openai/gpt-oss-20b',
 'modelo_embeddings': 'sentence-transformers/paraphrase-MiniLM-L6-v2',
 'documentos_no_indice': 537}

In [98]:
bot_rag.clear_documents()

Documentos resolvidos


In [99]:
bot_rag.get_stats()

{'documentos_carregados': False,
 'mensagens_no_historico': 7,
 'modelo_llm': 'openai/gpt-oss-20b',
 'modelo_embeddings': 'sentence-transformers/paraphrase-MiniLM-L6-v2'}